In [1]:
import cv2
import depthai as dai

# Define the pipeline
pipeline = dai.Pipeline()

# Define RGB camera node
rgb_cam = pipeline.createColorCamera()
rgb_cam.setBoardSocket(dai.CameraBoardSocket.RGB)
rgb_cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

# Create output for RGB
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
rgb_cam.video.link(xout_rgb.input)

# Start the pipeline
with dai.Device(pipeline) as device:
    # Output queues
    q_rgb = device.getOutputQueue(name="rgb", maxSize=1, blocking=False)

    # Get RGB frames
    while True:
        in_rgb = q_rgb.get()
        if in_rgb is not None:
            # Get the RGB frame
            frame = in_rgb.getCvFrame()
            
            # Display the RGB frame
            cv2.imshow("RGB", frame)

            # Save the frame (replace with your desired path)
            cv2.imwrite("real_world_object.jpg", frame)

            # Break the loop after saving the image
            break

            # Exit when 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

# Destroy all OpenCV windows
cv2.destroyAllWindows()


/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_50964/2865065955.py:9: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  rgb_cam.setBoardSocket(dai.CameraBoardSocket.RGB)


RuntimeError: No available devices

In [ ]:
import numpy as np
from math import sqrt

# Load rotational matrix
rot_matrix = np.loadtxt('rgb_rotation_matrix.txt')

print("ROTATIONAL MATRIX:")
print(rot_matrix)

# Load translation vector
tra_v = []
with open('rgb_translation_vector.txt', 'r') as f:
    for line in f:
        tra_v.append([float(num) for num in line.split()])

np_tra_v = np.array(tra_v)

print("\nTRANSLATION VECTOR:")
print(np_tra_v)

# Load camera intrinsic matrix
c_mtx = []
with open('rgb_camera_matrix.txt', 'r') as f:
    for line in f:
        c_mtx.append([float(num) for num in line.split()])

print("\nCAMERA INTRINSIC MATRIX:")
print(c_mtx)

# Convert camera intrinsic matrix to homogeneous form
int_mtx = np.append(np.append(c_mtx, [[0],[0],[1]], axis=1), [np.array([0,0,0,1])], axis=0)
print('\nIntrinsic matrix (homogeneous form):')
print(int_mtx)

# Combine rotation matrix with translation vector to form the extrinsic matrix
ext_mtx = np.hstack((rot_matrix, np_tra_v))

# Add the bottom row [0, 0, 0, 1] for homogeneous coordinates
ext_mtx = np.vstack((ext_mtx, [0, 0, 0, 1]))

print('\nEXTRINSIC MATRIX:')
print(ext_mtx)

# Calculate camera matrix
camera_matrix = np.dot(int_mtx, ext_mtx)
print("\nCAMERA MATRIX: ")
print(camera_matrix)

# Calculate the inverse of the camera matrix
inverse_mat = -np.linalg.inv(camera_matrix) 

# Define the points in homogeneous coordinates [X, Y, Z, 1]
project_points1 = np.array([[5],[10],[30],[1]])
project_points2 = np.array([[100],[90],[30],[1]])

# Transform points from camera frame to real-world frame
real_dim_p1 = inverse_mat.dot(project_points1)
real_dim_p2 = inverse_mat.dot(project_points2)

print("\nReal-world dimensions:")
print("Point 1:")
print("x axis length - ", real_dim_p1[0][0])
print("y axis length - ", real_dim_p1[1][0])
print("z axis length - ", real_dim_p1[2][0])

print("\nPoint 2:")
print("x axis length - ", real_dim_p2[0][0])
print("y axis length - ", real_dim_p2[1][0])
print("z axis length - ", real_dim_p2[2][0])

# Calculate the distance between the two points
dist = sqrt((real_dim_p2[0][0]-real_dim_p1[0][0])**2 +
            (real_dim_p2[1][0]-real_dim_p1[1][0])**2 +
            (real_dim_p2[2][0]-real_dim_p1[2][0])**2 )

print("\nDistance between the points:", dist*100, "cm")
